In [8]:
import pandas as pd


# Load IoT sensor data from CSV (Generated in Homework 1)
df = pd.read_csv("Graph.csv")


# Display the first few rows
print(df.head())

from web3 import Web3

# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

# Replace with actual contract address from Remix
contract_address = "0x42eE51261F08316AF0A3bEF516dA396d351F115E"

# Paste the ABI from Remix
abi = [
	{
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "patient_id",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "heart_rate",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "blood_pressure",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "oxygen_level",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "body_temp",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_patient_id",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_heart_rate",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_blood_pressure",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_oxygen_level",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_body_temp",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "patient_id",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "heart_rate",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "blood_pressure",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "oxygen_level",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "body_temp",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]  # Replace with your contract ABI


# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=abi)


# Set default sender to contract owner (required for authorization)
owner_address = contract.functions.owner().call()
web3.eth.default_account = owner_address

print(f"✅ Connected to Smart Contract at {contract_address} as owner {owner_address}")

import time

def send_iot_data(patient_id, heart_rate, blood_pressure, oxygen_level, body_temp):
    """Sends IoT data to the deployed smart contract"""
    txn = contract.functions.storeData(patient_id, heart_rate, blood_pressure, oxygen_level, body_temp).transact({
        'from': web3.eth.default_account,
        'gas': 3000000
    })
   
    receipt = web3.eth.wait_for_transaction_receipt(txn)
    print(f"✅ Data Stored: {patient_id} - {heart_rate} - {blood_pressure} - {oxygen_level} - {body_temp}, Txn Hash: {receipt.transactionHash.hex()}")

# Get current total records and max entries from contract
current_records = contract.functions.getTotalRecords().call()
max_entries = contract.functions.MAX_ENTRIES().call()

# Only send up to the allowed number of records
records_to_send = min(len(df), max_entries - current_records)
if records_to_send <= 0:
	print("⚠️ Storage limit reached. No new records will be sent.")
else:
	for i, (_, row) in enumerate(df.iterrows()):
		if i >= records_to_send:
			print("⚠️ Reached contract storage limit, stopping upload.")
			break
		send_iot_data(str(row["patient_id"]), str(row["heart_rate"]), str(row["blood_pressure"]), str(row["oxygen_level"]), str(row["body_temp"]))
		time.sleep(1)  # Delay to prevent flooding transactions

total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")

record = contract.functions.getRecord(0).call()
print("First Stored Record:", record)

import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "patient_id": record[1],
        "heart_rate": record[2],
        "blood_pressure": record[3],
		"oxygen_level": record[4],
		"body_temp": record[5]
    })


# Convert to a DataFrame
df = pd.DataFrame(data)


# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")


# Display first few records
print(df.head())

import numpy as np


# Extract numeric values from 'heart_rate' where applicable
df["numeric_value"] = df["patient_id"].str.extract(r'(\d+\.?\d*)').astype(float)

# Handle missing values (if any)
df.fillna(0, inplace=True)


# Display cleaned data
print(df.head())

# Save cleaned IoT data to a CSV file
df.to_csv("cleaned_iot_data1.csv", index=False)


print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

import numpy as np


# Extract numeric values from 'data_value' where applicable
df["numeric_value"] = df["heart_rate"].str.extract(r'(\d+\.?\d*)').astype(float)
df["numeric_value1"] = df["oxygen_level"].str.extract(r'(\d+\.?\d*)').astype(float)
df["numeric_value2"] = df["body_temp"].str.extract(r'(\d+\.?\d*)').astype(float)

# Handle missing values (if any)
df.fillna(0, inplace=True)


# Display cleaned data
print(df.head())

# Save cleaned IoT data to a CSV file
df.to_csv("cleaned_iot_data1.csv", index=False)


print("✅ Cleaned IoT data saved successfully as cleaned_iot_data1.csv")



         timestamp patient_id  heart_rate blood_pressure  oxygen_level  \
0  05/06/2025 0:01     PAT143          72         100/87            96   
1  05/07/2025 1:01     PAT143          90         117/83            96   
2  05/08/2025 2:01     PAT143          68         125/84            99   
3  05/09/2025 3:01     PAT143          72         123/79            97   
4  05/10/2025 4:01     PAT143          85         109/89            95   

   body_temp  
0       36.3  
1       36.9  
2       37.3  
3       37.9  
4       36.1  
✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0x42eE51261F08316AF0A3bEF516dA396d351F115E as owner 0x56bA4c12932a186922977AE1E6E167F57215512F
⚠️ Storage limit reached. No new records will be sent.
Total IoT records stored: 100
First Stored Record: [1749478546, 'PAT143', '72', '100/87', '96', '36.3']
            timestamp patient_id heart_rate blood_pressure oxygen_level  \
0 2025-06-09 14:15:46     PAT143         72         100/87        